## Import packages used

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import midasmlpy.date_functions as datef # used to handle different frequencies of data and to create lags
# import midasmlpy.sparse_group_lasso as sgl # used to run the sparse group lasso and related functions
from sklearn.model_selection import train_test_split

In [3]:
import sglfitF

## Load data

Load data from excel

In [4]:
# load data from xlsx files and create a dataframe
Predictors = pd.read_excel('/Users/m.egelundmuller/Documents/GitHub/midasmlpy/user_guide/predictors-monthly.xlsx').to_numpy()
Target = pd.read_excel('/Users/m.egelundmuller/Documents/GitHub/midasmlpy/user_guide/gdp-quarterly.xlsx').to_numpy()

Split data into dates and data tables

In [5]:
# Y data and X and Y dates can also be defined as they are the same for all iterations
Y_date = Target[:,0]
Y = Target[:,1]
X_date = Predictors[:,0]
X = Predictors[:,1:]

## Transform data using functions from data_functions

Define variables ued in transformation

In [6]:
# Lag variables
x_lags = 3
y_lags = 0
horizon = 0

# Legendre matrix
degree = 4 # 3 degrees + polynomial 0

Call data transformation function

In [7]:
transformed_data = datef.data_transform(Y, Y_date, X, X_date, x_lags, y_lags, horizon, degree=degree, standardize = True)

In [8]:
x = transformed_data['X_tilde']
y = transformed_data['Y']

# # Split x and y into a 80/20 train test split
train_size = int(0.8*x.shape[0])
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## sgLasso

In [15]:
import numpy as np
import sparsegllog_compiled # the sparse group lasso module from fortran
from scipy.sparse.linalg import svds
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedKFold, KFold
import random
random.seed(111)

########################################################################################

########### Functions related to fitting the sparse group lasso model.##################

########################################################################################

def calc_gamma(x, ix, iy, bn):
    """
    Calculates a measure (gamma) for columns of matrix 'x' specified by ranges in 'ix' and 'iy'.
    """
    gamma = np.full(bn, np.nan)
    for g in range(bn):
        grabcols = slice(ix[g], iy[g] + 1)  # Python uses 0-based indexing
        submatrix = x[:, grabcols]
        ncols = submatrix.shape[1]
        
        if ncols > 2:
            # Calculate the largest singular value squared 
            singular_values = svds(submatrix, k=1, return_singular_vectors=False, random_state = 42)
            gamma[g] = singular_values[0]**2
        elif ncols == 2:
            # Returns the largest squared singular value of a n-by-2 matrix x
            mat = np.dot(submatrix.T, submatrix)
            tr = mat[0, 0] + mat[1, 1]
            det = mat[0, 0] * mat[1, 1] - mat[0, 1] * mat[1, 0] 
            gamma[g] = (tr + np.sqrt(tr**2 - 4 * det)) / 2
        else:
            gamma[g] = np.sum(submatrix**2)
    
    return gamma / x.shape[0]


def sgLasso_estimation(x, y, group_size, alsparse, family, pmax = 100, intr = True, nlam=None, ulam=None):
    """
    Implements the Sparse Group Lasso algorithm, which is a regularization technique combining 
    both lasso (L1) and group lasso (L2) penalties. This method is particularly useful for 
    scenarios where both individual feature sparsity and group sparsity are desired. The 
    algorithm assumes that features are divided into groups of equal size.

    Args:
        x (numpy.ndarray): A (n_obs, n_features) matrix of input features, where n_obs is the number of observations and 
                           n_features is the total number of features.
        y (numpy.ndarray): A vector of length n_obs that contains the dependent variable (response variable).
        group_size (int): The number of features in each group. It is assumed that all groups have the same number of features.
        alsparse (float): The alpha parameter that balances the L1 and L2 penalties. An alsparse close to 1.0 gives more weight to the 
                          L1 part (similar to Lasso), while an alsparse close to 0.0 gives more weight to the L2 part (similar to Group Lasso).
        nlam (int, optional): The number of lambda values to use for fitting the model. Default is 100.
        ulam (numpy.ndarray, optional): A sequence of lambda values to use for fitting the model. Default is np.ones(nlam).
        pmax (int, optional): The maximum number of non-zero coefficients allowed in the model. Default is 100.
        intr (bool, optional): If True, an intercept is included in the model. Default is True.

    Returns:
        tuple: Contains the following elements:
            nalam (int): The number of lambda values actually used.
            b0 (numpy.ndarray): The estimated intercept (if intr is True).
            beta (numpy.ndarray): Coefficient estimates for the predictors.
            activeGroup (numpy.ndarray): Indices of active groups in the model.
            nbeta (int): Number of active predictors.
            alam (numpy.ndarray): The sequence of lambda values used.
            npass (int): The number of passes (iterations) over the data.
            jerr (int): An error code (if any) from the fitting process (0 means no error).
    """
    if ulam is not None:
        ulam = np.array(ulam)
        nlam = len(ulam)  # Override nlam based on the length of ulam if ulam is provided
    elif nlam is None:
        nlam = 100  # Default value if neither ulam nor nlam is provided

    if ulam is None:
        ulam = np.ones(nlam)  # Default ulam if not provided

    nobs,nvars = x.shape[0], x.shape[1] # Number of observations and features
    eps = 1e-8 # Convergence threshold
    maxit = 1000000 # Maximum number of iterations
    bn = x.shape[1]//group_size # Number of groups as an integer
    bs = np.full(bn, group_size, dtype=int) # Elements in groups
    ix, iy =  np.array(range(0, nvars, group_size)), np.array(range(group_size-1, nvars, group_size)) # Placement og first column of each group in x
    pf, pfl1 = np.sqrt(bs),np.ones(nvars) # Penalty factors for L2 and L1 penalties
    dfmax = bn + 1 # Maximum number of groups
    flmin = 0.01 if nobs < nvars else 1e-04
    lb,ub = np.full(bn, -np.inf),np.full(bn, np.inf) # Lower and upper bounds for the coefficients
    
    if family == 'binomial':
        gam = 0.25 * calc_gamma(x, ix, iy, bn) # Calculate gamma values for each group of features (columns) 
        _nalam, b0, beta, _activeGroup, _nbeta, alam, npass, jerr = sparsegllog_compiled.log_sparse_four(x = x,
                        y = y, bn = bn, bs = bs, 
                        ix = ix + 1, iy = iy + 1, # iy and ix are +1 as fortran is index 1 while python is index 0
                        gam = gam, nobs = nobs, 
                        nvars = nvars, pf = pf, pfl1 = pfl1, dfmax = dfmax, pmax = pmax, 
                        nlam = nlam, flmin = flmin, ulam = ulam, eps = eps, maxit = maxit, 
                        intr = intr, lb = lb, ub = ub, alsparse = alsparse)
        mse = None # to make it easier to return the same number of variables for all families
    if family == 'gaussian':
        if intr:
            y = y-y.mean()
        gam = calc_gamma(x, ix, iy, bn) # Calculate gamma values for each group of features (columns) 
        _nalam, b0, beta, _activeGroup, _nbeta, alam, npass, jerr, mse = sparsegllog_compiled.sparse_four(x = x,
                        y = y, bn = bn, bs = bs, 
                        ix = ix + 1, iy = iy + 1, # iy and ix are +1 as fortran is index 1 while python is index 0
                        gam = gam, nobs = nobs, 
                        nvars = nvars, pf = pf, pfl1 = pfl1, dfmax = dfmax, pmax = pmax, 
                        nlam = nlam, flmin = flmin, ulam = ulam, eps = eps, maxit = maxit, 
                        intr = intr, lb = lb, ub = ub, alsparse = alsparse)
    if jerr != 0:
        raise ValueError("Error in the sparse group lasso estimation.")
    if npass == maxit:
        raise ValueError("Failed to converge in the sparse group lasso estimation.")
    return b0, beta, alam, npass, jerr, mse

########################################################################################

######### Functions related to finding the optimal sparse group lasso model.############

########################################################################################

def evaluate_binomials(x, y, b0, beta,eval = 'auc', threshold=0.5):
    """
    Evaluate the performance of several logistic regression models using specified metrics for different values of lambda.

    Parameters:
    - x (ndarray): A 2D numpy array of input features (identical to `x` in `predict`).
    - y (ndarray): A 1D numpy array containing the true binary outcomes (0 or 1) for each sample in `x`.
    - b0 (ndarray): A 1D numpy array of intercepts, one for each model being evaluated.
    - beta (ndarray): A 2D numpy array where each column corresponds to the coefficients of a model.
    - eval (str): The metric for evaluation; 'accuracy' for accuracy score, 'auc' for AUC score.

    Returns:
    - accuracies (list): If `eval` == 'accuracy', a list of accuracy scores for each model.
    - auc_scores (list): If `eval` == 'auc', a list of AUC scores for each model.
    """
    evaluation_score = [0] * len(b0)  # this will store evaluation score
    for l in range(len(b0)):
        probabilities = 1 / (1 + np.exp(-np.dot(x, beta[:,l]) + b0[l]))
        predictions = (probabilities > threshold).astype(int)
        if eval == 'accuracy':
            evaluation_score[l] = accuracy_score(y, predictions)  
        elif eval == 'auc':
            evaluation_score[l] = roc_auc_score(y, predictions)
        else:
            raise ValueError("Invalid evaluation metric. Use 'accuracy' or 'auc'.")
    return evaluation_score



def evaluate_gaussian(x, y, b0, beta, intr, eval='mse'):
    """
    Evaluate the performance of several linear regression models using specified metrics.

    Parameters:
    - x (ndarray): A 2D numpy array of input features (identical to `x` in `predict`).
    - y (ndarray): A 1D numpy array containing the true continuous outcomes for each sample in `x`.
    - b0 (ndarray): A 1D numpy array of intercepts, one for each model being evaluated.
    - beta (ndarray): A 2D numpy array where each column corresponds to the coefficients of a model.
    - eval (str): The metric for evaluation; options are 'mse' for Mean Squared Error or 'r2' for R-squared.

    Returns:
    - evaluation_scores (list): A list of scores, either MSE or R-squared, for each model.
    """
    evaluation_scores = [0] * len(b0)  # this will store evaluation scores
    for l in range(len(b0)):
        predictions = np.dot(x, beta[:,l]) + b0[l]
        if intr:  # Adjust predictions if intercept was used during fitting
            predictions += mean_y
        if eval == 'mse':
            evaluation_scores[l] = mean_squared_error(y, predictions)
        elif eval == 'r2':
            evaluation_scores[l] = r2_score(y, predictions)
        else:
            raise ValueError("Invalid evaluation metric. Use 'mse' or 'r2'.")
    return evaluation_scores


def best_lambda_find(x,y,group_size, alsparse, family, nlam = 100, pmax = 100, intr = True,k_folds = 5):
    """
    Find the best model using sparse group lasso. The sparse group lasso finds coefficients for nlam values of lambda, and the best model
    is chosen as the one with the highest mean performance in k-fold cross-validation.

    Parameters:
    - x (ndarray): A 2D numpy array of input features (identical to `x` in `predict`).
    - y (ndarray): A 1D numpy array containing the true binary outcomes (0 or 1) for each sample in `x`.
    - group_size (int): The number of lags used for the legendre polynomials.
    - alsparse (int): The balancing parameter alpha.
    - nlam (int, optional): The number of lambda values to evaluate (defaults to 100).
    - pmax (int, optional): The maximum number of non-zero coefficients allowed in the model (defaults to 100).
    - intr (bool, optional): If True, an intercept is included in the model (defaults to True).
    - k_folds (int, optional): The number of folds for cross-validation (defaults to 5).

    Returns:
    - best_model (dict): A dictionary containing the following
        - 'b0' (float): The intercept of the best model.
        - 'beta' (ndarray): The coefficients of the best model.
        - 'maximized_performance' (float): The maximized performance of the best model.
        - 'best_lambda' (float): The lambda value of the best model.
    """
    # Find model nlam number of models
    b0, beta, alam, _npass, _jerr, mse = sgLasso_estimation(x, y, group_size, alsparse,family, pmax, intr)

    # Find mean performance for each lambda
    # Split the data into k_folds
    if family == 'binomial':
        kf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
    if family == 'gaussian':   
        kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    # initialize performance list
    performance = []
    for train_index, test_index in kf.split(x,y):
        # Based on the split, create the training and test data for this fold
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Estimate the model on the training data
        b0test, betatest, _alam, _npass, _jerr, msetrain = sgLasso_estimation(x_train, y_train, group_size, alsparse, family, pmax = pmax, intr = intr, ulam = alam)
        if family == 'gaussian':
            performance.append(evaluate_gaussian(x_test, y_test, b0test, betatest,intr,eval = 'mse'))
        if family == 'binomial':
            performance.append(evaluate_binomials(x_test, y_test, b0test, betatest,eval = 'auc', threshold=0.5))

    performance = np.array(performance)
    mean_performance = np.mean(performance, axis=0)
    if family == 'gaussian':
        best_lambda = np.argmin(mean_performance)
    if family == 'binomial':
        best_lambda = np.argmax(mean_performance)
    return {'b0': b0[best_lambda], 
            'beta': beta[:,best_lambda], 
            'best_performance': mean_performance[best_lambda], 
            'best_lambda': alam[best_lambda]}
    
def best_model(x, y, group_size, family, nlam = 100, pmax = 100, intr = True, k_folds = 5, disp_flag = True, alpha_values = None, alpha = None):
    """
    Function to find the best model based on the maximized performance of the model. The function uses the bestlambda function to find the best lambda value for the model.

    Parameters:
    x: numpy array
        The predictors for the model
    y: numpy array
        The target variable for the model
    group_size: int
        The number of groups in the model
    nlam: int
        The number of lambda values to test
    pmax: int
        The maximum number of variables in the model
    intr: bool
        Whether to include the intercept in the model
    k_folds: int
        The number of folds to use in the cross-validation
    disp_flag: bool
        Whether to display the performance at different values of alpha
    alpha_values: int
        The number of alpha values to test
    alpha: list
        The alpha value to test

    Returns:
    dict
        A dictionary containing the best alpha value, the best performance, the intercept and the coefficients of the model
    """
    if alpha is not None:
        alsparse_values = np.array(alpha)
    elif alpha_values is not None:
        alsparse_values = np.linspace(1, 0, alpha_values)
    else:
        alsparse_values = np.linspace(1, 0, 5)

    # Dictionary to store the average maximized performances for each alsparse
    if disp_flag:
        performance_dict = {}
    best_performance = None
    best_alsparse = None
    b0, beta = None, None  # Initialize parameters that will store best model coefficients

    # Cross-validation process
    for alsparse in alsparse_values:
        model_result = best_lambda_find(x,y,group_size, alsparse, family, nlam = nlam, pmax = pmax, intr = intr,k_folds = k_folds)
        # Append the maximized performance of this fold
        if disp_flag:
            performance_dict[alsparse] = model_result['best_performance'].round(5)
        # If this fold has a higher maximized performance than the previous best, update the best performance
        if best_performance is None:
                best_performance = model_result['best_performance']
                best_alsparse = alsparse
                b0 = model_result['b0']
                beta = model_result['beta'] 
                best_lambda = model_result['best_lambda']
        else:
            if family == 'gaussian':
                if model_result['best_performance']<best_performance:
                    best_performance = model_result['best_performance']
                    best_alsparse = alsparse
                    b0 = model_result['b0']
                    beta = model_result['beta'] 
                    best_lambda = model_result['best_lambda']
            if family == 'binomial':
                if model_result['best_performance']>best_performance:
                    best_performance = model_result['best_performance']
                    best_alsparse = alsparse
                    b0 = model_result['b0']
                    beta = model_result['beta'] 
                    best_lambda = model_result['best_lambda']

    if disp_flag:
        print('The performance at different values of alpha are:')
        print(performance_dict)
    
    return {'best_alsparse':best_alsparse, 
            'best_performance':best_performance, 
            'b0':b0, 
            'beta':beta,
            'best_lambda': best_lambda}

In [10]:
model2 = best_model(x = x_train, y = y_train, group_size = degree, family = 'gaussian', nlam = 100, pmax = 122, intr = False, k_folds = 3, disp_flag = True, alpha_values = 11, alpha = None)

The performance at different values of alpha are:
{1.0: 12.0668, 0.9: 12.21292, 0.8: 12.31221, 0.7: 12.38778, 0.6: 12.4934, 0.5: 12.56949, 0.3999999999999999: 12.62666, 0.29999999999999993: 12.71384, 0.19999999999999996: 12.82012, 0.09999999999999998: 12.88895, 0.0: 12.95272}


In [11]:
model2['best_performance']

12.06679602547654

In [12]:
model2 = best_model(x = x_train, y = y_train, group_size = degree, family = 'gaussian', nlam = 100, pmax = 122, intr = False, k_folds = 3, disp_flag = True, alpha_values = 11, alpha = None)

The performance at different values of alpha are:
{1.0: 12.0668, 0.9: 12.21292, 0.8: 12.31221, 0.7: 12.38778, 0.6: 12.4934, 0.5: 12.56949, 0.3999999999999999: 12.62666, 0.29999999999999993: 12.71384, 0.19999999999999996: 12.82012, 0.09999999999999998: 12.88895, 0.0: 12.95272}


In [27]:
model2['best_performance']

12.06679602547654

In [13]:
x = x_train
y = y_train
group_size = degree
family = 'gaussian'
nlam = 100
pmax = 122
intr = False
k_folds = 3
disp_flag = True
alpha_values = 11
alpha = None
alsparse = 0.5

In [14]:
    # Find model nlam number of models
    b0, beta, alam, _npass, _jerr, mse = sgLasso_estimation(x, y, group_size, alsparse,family, pmax, intr)

    # Find mean performance for each lambda
    # Split the data into k_folds
    if family == 'binomial':
        kf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
    if family == 'gaussian':   
        kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    # initialize performance list
    performance = []
    for train_index, test_index in kf.split(x,y):
        # Based on the split, create the training and test data for this fold
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Estimate the model on the training data
        b0test, betatest, _alam, _npass, _jerr, msetrain = sgLasso_estimation(x_train, y_train, group_size, alsparse, family, pmax = pmax, intr = intr, ulam = alam)
        if family == 'gaussian':
            performance.append(evaluate_gaussian(x_test, y_test, b0test, betatest,eval = 'mse'))
        if family == 'binomial':
            performance.append(evaluate_binomials(x_test, y_test, b0test, betatest,eval = 'auc', threshold=0.5))

    performance = np.array(performance)
    mean_performance = np.mean(performance, axis=0)
    if family == 'binomial':
        best_lambda = np.argmax(mean_performance)
    if family == 'gaussian':
        best_lambda = np.argmin(mean_performance)

TypeError: evaluate_gaussian() missing 1 required positional argument: 'intr'

In [ ]:
mean_performance

array([15.72073806, 15.68469278, 15.49494481, 15.32664924, 15.1753788 ,
       15.0394381 , 14.91728718, 14.79831989, 14.69161439, 14.58669592,
       14.4925171 , 14.40975478, 14.32792115, 14.23311733, 14.0924809 ,
       13.97235306, 13.86846429, 13.77427229, 13.6950444 , 13.62711568,
       13.57304908, 13.52009265, 13.45608846, 13.37643942, 13.3041933 ,
       13.24390963, 13.20272572, 13.17317758, 13.16261396, 13.16207891,
       13.15776153, 13.15411543, 13.15249136, 13.13512304, 13.11654678,
       13.09464479, 13.05082401, 12.98869776, 12.91041804, 12.84006636,
       12.77987338, 12.7202505 , 12.6619418 , 12.61368418, 12.58416023,
       12.56948842, 12.58089292, 12.60514016, 12.64270585, 12.68403573,
       12.74605675, 12.81651275, 12.9162938 , 13.02654741, 13.14018596,
       13.24087544, 13.3647866 , 13.50583955, 13.65282761, 13.81935746,
       13.99596268, 14.17420344, 14.36136385, 14.54756812, 14.71971168,
       14.88544037, 15.07012714, 15.26097823, 15.45872708, 15.66

In [ ]:
mean_performance[best_lambda]

12.569488420636413

In [ ]:
np.argmin(mean_performance)

45

In [22]:
x = transformed_data['X_tilde']
y = transformed_data['Y']

# # Split x and y into a 80/20 train test split
train_size = int(0.8*x.shape[0])
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [23]:
b0, beta, alam, _npass, _jerr, mse = sgLasso_estimation(x_train, y_train, group_size, alsparse,family, pmax, intr)
evaluation_scores = [0] * len(b0)  # this will store evaluation scores
for l in range(len(b0)):
    predictions = np.dot(x_test, beta[:,l]) + b0[l]
    evaluation_scores[l] = mean_squared_error(y_test, predictions)


In [26]:
evaluation_scores

[66.67973619271427,
 66.66931066055061,
 66.59851725606049,
 66.5317382351875,
 66.46871618002568,
 66.40920931545486,
 66.3529970300571,
 66.29987845217595,
 66.24967026451827,
 66.20220621963709,
 66.15733641090557,
 66.1149261202311,
 66.07485511895634,
 66.03701712057271,
 66.00131938223261,
 65.95432307928232,
 65.902637478755,
 65.86062823728464,
 65.81928510611839,
 65.77666324237987,
 65.72177092426341,
 65.71595239753776,
 65.74122524773539,
 65.76844263013173,
 65.71892815299486,
 65.67069136843438,
 65.63721084185885,
 65.61596415275122,
 65.64115886486319,
 65.67454284698647,
 65.82231958955516,
 66.00603189368738,
 66.15102225346607,
 66.25443014565182,
 66.32635088372936,
 66.41786136425802,
 66.41424197901995,
 66.39330802493555,
 66.43509613506544,
 66.5046442677282,
 66.58454308107342,
 66.66999263480321,
 66.75863384605294,
 66.851708892422,
 67.00356943408615,
 67.19962874767405,
 67.40932367352032,
 67.64931674004487,
 67.91035460480109,
 68.18692489683862,
 68.4656

In [24]:
pd.DataFrame(predictions,y_test)

,0
3.58582,1.033435
2.46998,5.012366
1.59778,2.844964
0.73724,6.968606
2.31139,10.076522
1.28244,5.805969
2.82754,5.754215
2.21107,3.512108
1.94283,4.712126
2.23324,7.100055


In [ ]:
pd.DataFrame(evaluation_scores)

,0
0,15.726799
1,15.695337
2,15.484573
3,15.291053
4,15.113180
...,...
95,0.482967
96,0.451148
97,0.421517
98,0.393788
